In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold


In [2]:
my_images = []
labels = [0]*5 + [1]*5  # 내 얼굴 1, 타인 0

for i in range(10):
    file = f"./my_images/img{i+1:02d}.jpg"
    image = cv.imread(file)
    image = cv.resize(image, (96, 96))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    my_images.append(image)

X = np.array(my_images, dtype='float32') / 255.0
y = np.array(labels)


In [3]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7,1.3],
    fill_mode='nearest'
)


In [4]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
models = []

for train_index, val_index in kf.split(X):
    print(f"\n--- Fold {fold} ---")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    datagen.fit(X_train)

    # CNN 모델 정의
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(96,96,3)),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2,2)),

        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2,2)),

        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Flatten(),

        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=2),
        validation_data=(X_val, y_val),
        epochs=200,
        callbacks=[early_stop],
        verbose=1
    )

    models.append(model)
    fold += 1



--- Fold 1 ---
Epoch 1/200


C:\Users\yhye1\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6868
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6868
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6250 - loss: 0.7508 - val_accuracy: 0.5000 - val_loss: 0.6909
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.3750 - loss: 0.9992 - val_accuracy: 0.5000 - val_loss: 0.6995
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.2500 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.7402
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6250 - loss: 0.8944 - val_accuracy: 0.5000 - val_loss: 0.8184
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6250 - loss: 0.6929 - val_accuracy: 0.5000 - val_loss: 0.8793
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6250 - loss: 0.6157 - val_accuracy: 0.5000 - val_loss: 0.9340
Epo

In [5]:
test_images = []
for i in range(10):
    file = f"./test_images/img{i+1:02d}.jpg"
    image = cv.imread(file)
    image = cv.resize(image, (96, 96))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    test_images.append(image)

test_images_arr = np.array(test_images, dtype='float32') / 255.0


In [6]:
predictions = np.zeros((len(test_images_arr), 1))
for model in models:
    predictions += model.predict(test_images_arr)

predictions /= len(models)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


In [7]:
for i, p in enumerate(predictions):
    if p > 0.5:
        print(f"img{i+1:02d}.jpg → ✅ 내 얼굴 ({p[0]:.3f})")
    else:
        print(f"img{i+1:02d}.jpg → ❌ 타인 ({p[0]:.3f})")


img01.jpg → ❌ 타인 (0.488)
img02.jpg → ❌ 타인 (0.485)
img03.jpg → ❌ 타인 (0.481)
img04.jpg → ❌ 타인 (0.488)
img05.jpg → ❌ 타인 (0.485)
img06.jpg → ❌ 타인 (0.482)
img07.jpg → ❌ 타인 (0.482)
img08.jpg → ❌ 타인 (0.483)
img09.jpg → ❌ 타인 (0.487)
img10.jpg → ❌ 타인 (0.482)
